# Information Retrival
## Assignment 2, Group 42
### Question 2
#### Team Members: Harsh Bandhey, Md Talib, Arshdeep Singh

-----------------------------------

In [1]:
import os
import re
import nltk
import operator
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from collections import Counter

In [2]:
# nltk.download('wordnet')
# nltk.download('stopwords')

In [3]:
files = list()
for dirpath, dirs, fls in os.walk('stories'):  
    for filename in fls: 
        path = os.path.join(dirpath,filename)
        fname = os.path.join('',filename)
        if not fname.startswith(".") and not fname.endswith("html"):
            files.append(path)
print(len(files))

467


In [4]:
stop_words = set(stopwords.words('english'))
def preprocess(text):        
    text = text.lower() #1
    text = re.sub(r'[^\w\s]', ' ', text) #4
    tokens = word_tokenize(text) #2
    tokens = [word for word in tokens if word not in stop_words] #3
    tokens = [token.translate({ ord(c): None for c in "_'" }) for token in tokens] #Extrastep
    tokens = [token for token in tokens if not (token.strip()=='' or len(token)<3)] #5
    return tokens

In [5]:
def ProcessDocs():
    processed_docs = dict()
    for i in tqdm(range(len(files))):
        with open(files[i],encoding='unicode-escape') as f:
            text = f.read()
        words = preprocess(text)
        processed_docs[files[i]] = words
    return processed_docs
processed_docs = ProcessDocs()

100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [00:30<00:00, 15.26it/s]


#### Jaccard Coefficient

In [6]:
def unionfn(list1,list2):
    union = list(set(list1) | set(list2))
    return union
def intersectionfn(list1,list2):
    intersection = list(set(list1) & set(list2))
    return intersection

In [7]:
def JaccardScores(query):
    jaccard_scores = dict()
    for i in tqdm(range(len(files))):
        try:
            words = processed_docs[files[i]]
        except:
            with open(files[i],encoding='unicode-escape') as f:
                text = f.read()
            words = preprocess(text)
        union = unionfn(words,query)
        intersection = intersectionfn(words,query)
        jaccard_scores[files[i]] = len(intersection)/len(union) 
        jaccard_sorted = list(sorted(jaccard_scores.items(),key=operator.itemgetter(1),reverse=True))
    return jaccard_sorted

In [8]:
def JaccardSearchDocs(query,k=10):
    query =  preprocess(query)
#     query = [ps.stem(word) for word in query]
    print("Processed Query: ",query)
    jaccard_scores = JaccardScores(query)
    top_docs = [file for file, score in jaccard_scores]
    print()
    print("Printing top",k,"matched documents")
    for doc in top_docs[:k]:
        print(doc)
    return top_docs

In [9]:
# top_docs = JaccardSearchDocs("good day")

In [10]:
query = input("Query: ")
k = int(input("No of top matches required: "))
found = JaccardSearchDocs(query,k)

Query: good day
No of top matches required: 10


 10%|████████▍                                                                       | 49/467 [00:00<00:00, 480.39it/s]

Processed Query:  ['good', 'day']


100%|███████████████████████████████████████████████████████████████████████████████| 467/467 [00:00<00:00, 617.73it/s]


Printing top 10 matched documents
stories\quarter.c16
stories\blasters.fic
stories\cameloto.hum
stories\bestwish
stories\foxnstrk.txt
stories\mouslion.txt
stories\aminegg.txt
stories\crabhern.txt
stories\write
stories\horswolf.txt


#### TF-IDF Matrix

In [11]:
rawcount_term_frequency = []
for i in tqdm(range(len(files))):
    dictionary_tf = dict(Counter(processed_docs[files[i]]))
    rawcount_term_frequency.append(dictionary_tf)
rawcount_term_frequency = pd.DataFrame(rawcount_term_frequency)
rawcount_term_frequency = rawcount_term_frequency.fillna(0)
rawcount_term_frequency

100%|██████████████████████████████████████████████████████████████████████████████| 467/467 [00:00<00:00, 1441.81it/s]


shareware  trial  project  freeware  need  support  continue  100  west  \
0          1.0    1.0      1.0       1.0   1.0      1.0       1.0  3.0   2.0   
1          0.0    0.0      0.0       0.0   0.0      0.0       0.0  0.0   0.0   
2          0.0    0.0      1.0       0.0   0.0      0.0       0.0  0.0   0.0   
3          0.0    0.0      0.0       0.0   3.0      0.0       0.0  0.0   0.0   
4          0.0    0.0      0.0       0.0   0.0      0.0       0.0  0.0   0.0   
..         ...    ...      ...       ...   ...      ...       ...  ...   ...   
462        0.0    0.0      0.0       0.0   1.0      0.0       0.0  0.0   0.0   
463        0.0    0.0      1.0       0.0   0.0      1.0       1.0  0.0   0.0   
464        0.0    0.0      0.0       0.0   2.0      0.0       0.0  0.0   0.0   
465        0.0    0.0      0.0       0.0   2.0      4.0       1.0  0.0   0.0   
466        0.0    0.0      0.0       0.0   0.0      1.0       0.0  1.0   0.0   

     north  ...  nosedive  rekindle  oblierate  surreneder  destructing  sozj  \
0     20.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
1      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
2      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
3      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
4      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
..     ...  ...       ...       ...        ...         ...          ...   ...   
462    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
463    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
464    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
465    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
466    0.0  ...       1.0       1.0        1.0         1.0          1.0  11.0   

     kolden  tghe  culo  explaing  
0       0.0   0.0   0.0       0.0  
1       0.0   0.0   0.0       0.0  
2       0.0   0.0   0.0       0.0  
3       0.0   0.0   0.0       0.0  
4       0.0   0.0   0.0       0.0  
..      ...   ...   ...       ...  
462     0.0   0.0   0.0       0.0  
463     0.0   0.0   0.0       0.0  
464     0.0   0.0   0.0       0.0  
465     0.0   0.0   0.0       0.0  
466     1.0   1.0   1.0       1.0  

[467 rows x 50172 columns]

In [12]:
# rawcount_term_frequency.describe()

In [13]:
natural_term_frequency = pd.DataFrame(rawcount_term_frequency)
natural_term_frequency = natural_term_frequency.div(natural_term_frequency.sum(axis=1), axis=0)
natural_term_frequency

shareware     trial   project  freeware      need   support  continue  \
0     0.000585  0.000585  0.000585  0.000585  0.000585  0.000585  0.000585   
1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.002110  0.000000  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.000000  0.002346  0.000000  0.000000   
4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
..         ...       ...       ...       ...       ...       ...       ...   
462   0.000000  0.000000  0.000000  0.000000  0.001295  0.000000  0.000000   
463   0.000000  0.000000  0.000304  0.000000  0.000000  0.000304  0.000304   
464   0.000000  0.000000  0.000000  0.000000  0.001105  0.000000  0.000000   
465   0.000000  0.000000  0.000000  0.000000  0.000672  0.001344  0.000336   
466   0.000000  0.000000  0.000000  0.000000  0.000000  0.000547  0.000000   

          100      west    north  ...  nosedive  rekindle  oblierate  \
0    0.001756  0.001171  0.01171  ...  0.000000  0.000000   0.000000   
1    0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
2    0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
3    0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
4    0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
..        ...       ...      ...  ...       ...       ...        ...   
462  0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
463  0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
464  0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
465  0.000000  0.000000  0.00000  ...  0.000000  0.000000   0.000000   
466  0.000547  0.000000  0.00000  ...  0.000547  0.000547   0.000547   

     surreneder  destructing      sozj    kolden      tghe      culo  explaing  
0      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
1      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
3      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
..          ...          ...       ...       ...       ...       ...       ...  
462    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
463    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
464    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
465    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
466    0.000547     0.000547  0.006014  0.000547  0.000547  0.000547  0.000547  

[467 rows x 50172 columns]

In [14]:
# natural_term_frequency.describe()

In [15]:
log_term_frequency = np.log(pd.DataFrame(1+rawcount_term_frequency))
log_term_frequency

shareware     trial   project  freeware      need   support  continue  \
0     0.693147  0.693147  0.693147  0.693147  0.693147  0.693147  0.693147   
1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.693147  0.000000  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.000000  1.386294  0.000000  0.000000   
4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
..         ...       ...       ...       ...       ...       ...       ...   
462   0.000000  0.000000  0.000000  0.000000  0.693147  0.000000  0.000000   
463   0.000000  0.000000  0.693147  0.000000  0.000000  0.693147  0.693147   
464   0.000000  0.000000  0.000000  0.000000  1.098612  0.000000  0.000000   
465   0.000000  0.000000  0.000000  0.000000  1.098612  1.609438  0.693147   
466   0.000000  0.000000  0.000000  0.000000  0.000000  0.693147  0.000000   

          100      west     north  ...  nosedive  rekindle  oblierate  \
0    1.386294  1.098612  3.044522  ...  0.000000  0.000000   0.000000   
1    0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
2    0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
3    0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
4    0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
..        ...       ...       ...  ...       ...       ...        ...   
462  0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
463  0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
464  0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
465  0.000000  0.000000  0.000000  ...  0.000000  0.000000   0.000000   
466  0.693147  0.000000  0.000000  ...  0.693147  0.693147   0.693147   

     surreneder  destructing      sozj    kolden      tghe      culo  explaing  
0      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
1      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
3      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4      0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
..          ...          ...       ...       ...       ...       ...       ...  
462    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
463    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
464    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
465    0.000000     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
466    0.693147     0.693147  2.484907  0.693147  0.693147  0.693147  0.693147  

[467 rows x 50172 columns]

In [16]:
# log_term_frequency.describe()

In [17]:
rawcount_term_frequency

shareware  trial  project  freeware  need  support  continue  100  west  \
0          1.0    1.0      1.0       1.0   1.0      1.0       1.0  3.0   2.0   
1          0.0    0.0      0.0       0.0   0.0      0.0       0.0  0.0   0.0   
2          0.0    0.0      1.0       0.0   0.0      0.0       0.0  0.0   0.0   
3          0.0    0.0      0.0       0.0   3.0      0.0       0.0  0.0   0.0   
4          0.0    0.0      0.0       0.0   0.0      0.0       0.0  0.0   0.0   
..         ...    ...      ...       ...   ...      ...       ...  ...   ...   
462        0.0    0.0      0.0       0.0   1.0      0.0       0.0  0.0   0.0   
463        0.0    0.0      1.0       0.0   0.0      1.0       1.0  0.0   0.0   
464        0.0    0.0      0.0       0.0   2.0      0.0       0.0  0.0   0.0   
465        0.0    0.0      0.0       0.0   2.0      4.0       1.0  0.0   0.0   
466        0.0    0.0      0.0       0.0   0.0      1.0       0.0  1.0   0.0   

     north  ...  nosedive  rekindle  oblierate  surreneder  destructing  sozj  \
0     20.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
1      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
2      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
3      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
4      0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
..     ...  ...       ...       ...        ...         ...          ...   ...   
462    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
463    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
464    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
465    0.0  ...       0.0       0.0        0.0         0.0          0.0   0.0   
466    0.0  ...       1.0       1.0        1.0         1.0          1.0  11.0   

     kolden  tghe  culo  explaing  
0       0.0   0.0   0.0       0.0  
1       0.0   0.0   0.0       0.0  
2       0.0   0.0   0.0       0.0  
3       0.0   0.0   0.0       0.0  
4       0.0   0.0   0.0       0.0  
..      ...   ...   ...       ...  
462     0.0   0.0   0.0       0.0  
463     0.0   0.0   0.0       0.0  
464     0.0   0.0   0.0       0.0  
465     0.0   0.0   0.0       0.0  
466     1.0   1.0   1.0       1.0  

[467 rows x 50172 columns]

In [18]:
binary_term_frequency = rawcount_term_frequency.copy()
binary_term_frequency[binary_term_frequency!=0]=1
binary_term_frequency = binary_term_frequency.astype(int)
binary_term_frequency

shareware  trial  project  freeware  need  support  continue  100  west  \
0            1      1        1         1     1        1         1    1     1   
1            0      0        0         0     0        0         0    0     0   
2            0      0        1         0     0        0         0    0     0   
3            0      0        0         0     1        0         0    0     0   
4            0      0        0         0     0        0         0    0     0   
..         ...    ...      ...       ...   ...      ...       ...  ...   ...   
462          0      0        0         0     1        0         0    0     0   
463          0      0        1         0     0        1         1    0     0   
464          0      0        0         0     1        0         0    0     0   
465          0      0        0         0     1        1         1    0     0   
466          0      0        0         0     0        1         0    1     0   

     north  ...  nosedive  rekindle  oblierate  surreneder  destructing  sozj  \
0        1  ...         0         0          0           0            0     0   
1        0  ...         0         0          0           0            0     0   
2        0  ...         0         0          0           0            0     0   
3        0  ...         0         0          0           0            0     0   
4        0  ...         0         0          0           0            0     0   
..     ...  ...       ...       ...        ...         ...          ...   ...   
462      0  ...         0         0          0           0            0     0   
463      0  ...         0         0          0           0            0     0   
464      0  ...         0         0          0           0            0     0   
465      0  ...         0         0          0           0            0     0   
466      0  ...         1         1          1           1            1     1   

     kolden  tghe  culo  explaing  
0         0     0     0         0  
1         0     0     0         0  
2         0     0     0         0  
3         0     0     0         0  
4         0     0     0         0  
..      ...   ...   ...       ...  
462       0     0     0         0  
463       0     0     0         0  
464       0     0     0         0  
465       0     0     0         0  
466       1     1     1         1  

[467 rows x 50172 columns]

In [19]:
doublenormal_term_frequency = 0.5*(rawcount_term_frequency.div(rawcount_term_frequency.max(axis=1), axis=0))+0.5
doublenormal_term_frequency

shareware  trial   project  freeware      need   support  continue  \
0        0.525  0.525  0.525000     0.525  0.525000  0.525000  0.525000   
1        0.500  0.500  0.500000     0.500  0.500000  0.500000  0.500000   
2        0.500  0.500  0.541667     0.500  0.500000  0.500000  0.500000   
3        0.500  0.500  0.500000     0.500  0.568182  0.500000  0.500000   
4        0.500  0.500  0.500000     0.500  0.500000  0.500000  0.500000   
..         ...    ...       ...       ...       ...       ...       ...   
462      0.500  0.500  0.500000     0.500  0.522727  0.500000  0.500000   
463      0.500  0.500  0.509259     0.500  0.500000  0.509259  0.509259   
464      0.500  0.500  0.500000     0.500  0.545455  0.500000  0.500000   
465      0.500  0.500  0.500000     0.500  0.525641  0.551282  0.512821   
466      0.500  0.500  0.500000     0.500  0.500000  0.505556  0.500000   

          100  west  north  ...  nosedive  rekindle  oblierate  surreneder  \
0    0.575000  0.55    1.0  ...  0.500000  0.500000   0.500000    0.500000   
1    0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
2    0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
3    0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
4    0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
..        ...   ...    ...  ...       ...       ...        ...         ...   
462  0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
463  0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
464  0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
465  0.500000  0.50    0.5  ...  0.500000  0.500000   0.500000    0.500000   
466  0.505556  0.50    0.5  ...  0.505556  0.505556   0.505556    0.505556   

     destructing      sozj    kolden      tghe      culo  explaing  
0       0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
1       0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
2       0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
3       0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
4       0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
..           ...       ...       ...       ...       ...       ...  
462     0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
463     0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
464     0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
465     0.500000  0.500000  0.500000  0.500000  0.500000  0.500000  
466     0.505556  0.561111  0.505556  0.505556  0.505556  0.505556  

[467 rows x 50172 columns]

In [20]:
# doublenormal_term_frequency.describe()

In [21]:
document_frequency = np.log(rawcount_term_frequency.shape[0]/binary_term_frequency.sum()+1)

In [22]:
# pd.DataFrame(document_frequency)

In [23]:
binary_tfidf = binary_term_frequency.multiply(document_frequency)
rawcount_tfidf = rawcount_term_frequency.multiply(document_frequency)
natural_tdidf = natural_term_frequency.multiply(document_frequency)
log_tfidf = log_term_frequency.multiply(document_frequency)
dn_tfidf = doublenormal_term_frequency.multiply(document_frequency)

In [24]:
def tfidfScore(doc_tfidf,query):
    query = preprocess(query)
    rawcount_tf = []
    dictionary_tf = dict(Counter(query))
    rawcount_tf.append(dictionary_tf)
    query_raw_tf = pd.DataFrame(rawcount_tf)
    cols = intersectionfn(doc_tfidf.columns,query_raw_tf.columns)
    scoretfidf = doc_tfidf[cols]
    scoretfidf = scoretfidf.sum(axis=1).sort_values(ascending=False)
    return scoretfidf
# tfidfScore(natural_tdidf,'good day')

In [25]:
def getTopK(doc_tfidf,query,string,k=5):
    top_docs = tfidfScore(doc_tfidf,query).head(5)
    print("Top",k,"docs by",string,"TF-IDF matching score")
    for idx,score in top_docs.iteritems():
        print(files[idx],", Score: ",score)
    print()

In [26]:
def SearchDocsTfidf(query,k=5):
    getTopK(binary_tfidf,query,k,"Binary")
    getTopK(rawcount_tfidf,query,k,"Raw Count")
    getTopK(natural_tdidf,query,k,"Term Frequency")
    getTopK(log_tfidf,query,k,"Logarithmic")
    getTopK(dn_tfidf,query,k,"Double Normalize")

In [27]:
query = input("Query: ")
SearchDocsTfidf(query,k=5)

Query: good day
Top Binary docs by 5 TF-IDF matching score
stories\100west.txt , Score:  1.8524058234335237
stories\hop-frog.poe , Score:  1.8524058234335237
stories\immorti.hum , Score:  1.8524058234335237
stories\idi.hum , Score:  1.8524058234335237
stories\hound-b.txt , Score:  1.8524058234335237

Top Raw Count docs by 5 TF-IDF matching score
stories\gulliver.txt , Score:  186.68417688616
stories\vgilante.txt , Score:  136.05855374463943
stories\hound-b.txt , Score:  123.2884655845475
stories\outcast.dos , Score:  110.07327683758922
stories\aesop11.txt , Score:  101.26145033153476

Top Term Frequency docs by 5 TF-IDF matching score
stories\blossom.pom , Score:  0.038126620904131665
stories\contrad1.hum , Score:  0.03322451147229462
stories\blasters.fic , Score:  0.02693691409102778
stories\clevdonk.txt , Score:  0.02663256607273903
stories\pepdegener.txt , Score:  0.024501035354859174

Top Logarithmic docs by 5 TF-IDF matching score
stories\gulliver.txt , Score:  8.549192330562068
s

#### Cosine Similarity

In [28]:
import numpy.linalg as LA

In [29]:
query = 'good day'
query = preprocess(query)
rawcount_tf = []
dictionary_tf = dict(Counter(query))
rawcount_tf.append(dictionary_tf)
query_raw_tf = pd.DataFrame(columns= rawcount_term_frequency.columns)
temp_df = pd.DataFrame(rawcount_tf)
query_raw_tf = pd.merge(temp_df,query_raw_tf,how="left")
query_raw_tf = query_raw_tf[rawcount_term_frequency.columns]
query_raw_tf = query_raw_tf.fillna(0)
query_raw_tf

shareware  trial  project  freeware  need  support  continue  100  west  \
0          0      0        0         0     0        0         0    0     0   

   north  ...  nosedive  rekindle  oblierate  surreneder  destructing  sozj  \
0      0  ...         0         0          0           0            0     0   

   kolden  tghe  culo  explaing  
0       0     0     0         0  

[1 rows x 50172 columns]

In [30]:
def getCosineScores(doc_tfidf,query_tfidf):
    doc_scores = dict()
    cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)
    for index, vector in doc_tfidf.iterrows():
#         print(vector)
        for _, testV in query_tfidf.iterrows():
#             print(index)
            cosine = cx(vector, testV)
#             print(cosine)
            doc_scores[index]=cosine
    doc_sorted = list(sorted(doc_scores.items(),key=operator.itemgetter(1),reverse=True))
    return doc_sorted

In [31]:
def getTopKcosine(doc_tfidf,query_tfidf,string,k=5):
    top_docs = getCosineScores(doc_tfidf,query_tfidf)[:5]
    print("Top",k,"docs by",string,"TF-IDF matching score")
    for idx,score in top_docs:
        print(files[idx],", Score: ",score)
    print()

In [32]:
def SearchDocsTfidfCosine(query,k=5):
    
    query = preprocess(query)
    print("Processed Query:", query)
    rawcount_tf = []
    dictionary_tf = dict(Counter(query))
    rawcount_tf.append(dictionary_tf)
    query_raw_tf = pd.DataFrame(columns= rawcount_term_frequency.columns)
    temp_df = pd.DataFrame(rawcount_tf)
    query_raw_tf = pd.merge(temp_df,query_raw_tf,how="left")
    query_raw_tf = query_raw_tf[rawcount_term_frequency.columns]
    query_raw_tf = query_raw_tf.fillna(0)
    
    query_binary_tf = query_raw_tf.copy()
    query_binary_tf[query_binary_tf!=0]=1
    query_binary_tf = query_binary_tf.astype(int)
    
    query_natural_tf = query_raw_tf.copy()
    query_natural_tf = query_natural_tf.div(query_natural_tf.sum(axis=1), axis=0)
    
    query_log_tf = np.log(pd.DataFrame(1+query_raw_tf))
    
    query_dn_tf = 0.5*(query_raw_tf.div(query_raw_tf.max(axis=1), axis=0))+0.5
    print("Query TF Made")
    
    query_binary_tfidf = query_binary_tf.multiply(document_frequency)
    query_rawcount_tfidf = query_raw_tf.multiply(document_frequency)
    query_natural_tdidf = query_natural_tf.multiply(document_frequency)
    query_log_tfidf = query_log_tf.multiply(document_frequency)
    query_dn_tfidf = query_dn_tf.multiply(document_frequency)
    print("Query TF-IDF Made")
    
    print("Fecting Top Documents")
    print()
    getTopKcosine(binary_tfidf,query_binary_tfidf,k,"Binary")
    getTopKcosine(rawcount_tfidf,query_rawcount_tfidf,k,"Raw Count")
    getTopKcosine(natural_tdidf,query_natural_tdidf,k,"Term Frequency")
    getTopKcosine(log_tfidf,query_log_tfidf,k,"Logarithmic")
    getTopKcosine(dn_tfidf,query_dn_tfidf,k,"Double Normalize")

In [33]:
query = input("Query: ")
SearchDocsTfidfCosine(query,k=5)

Query: good day
Processed Query: ['good', 'day']
Query TF Made
Query TF-IDF Made
Fecting Top Documents

Top Binary docs by 5 TF-IDF matching score
stories\bestwish , Score:  0.055
stories\elveshoe.txt , Score:  0.049
stories\horswolf.txt , Score:  0.049
stories\mouslion.txt , Score:  0.049
stories\aminegg.txt , Score:  0.046

Top Raw Count docs by 5 TF-IDF matching score
stories\blossom.pom , Score:  0.103
stories\pepdegener.txt , Score:  0.094
stories\pepsi.degenerat , Score:  0.094
stories\brain.damage , Score:  0.092
stories\contrad1.hum , Score:  0.088

Top Term Frequency docs by 5 TF-IDF matching score
stories\blossom.pom , Score:  0.103
stories\pepdegener.txt , Score:  0.094
stories\pepsi.degenerat , Score:  0.094
stories\brain.damage , Score:  0.092
stories\contrad1.hum , Score:  0.088

Top Logarithmic docs by 5 TF-IDF matching score
stories\bestwish , Score:  0.082
stories\horswolf.txt , Score:  0.065
stories\mouslion.txt , Score:  0.062
stories\blossom.pom , Score:  0.061
stor